## Tutoriel de Données du Rover LEAD


**Tutoriel** : Ce tutoriel consiste en les étapes en Python pour extraire, visualiser et sauvegarder les données *(imagerie, LiDAR)* sous forme de fichiers rosbags capturés par le Rover Juno lors d'une réplication d'une mission lunaire.<br>
**Mission et Instrument** : Déploiement Analogique d'Exploration Lunaire (LEAD), Rover Juno <br>
**Exigences du système** : Python 3.8.8 <br>
**Niveau du tutoriel** : Intermédiaire <br>

***
**Licence MIT** <br>
Copyright (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence spatiale canadienne, 2023. <br>
Droit d'auteur (c) Sa Majesté le Roi du chef du Canada, représentée par l'Agence Spatiale Canadienne, 2023.<br>

Pour plus d'informations, veuillez vous référer au fichier *License.txt*. 

***
 **Informations contextuelles** 

Entre 2017 et 2019, l'Agence spatiale canadienne (ASC) s'est associée à l'Agence spatiale européenne (ESA) pour mener une série de tests sur le terrain afin de reproduire des scénarios d'une mission de retour d'échantillons lunaires. Ceci était pour acquérir des connaissances et une expérience pratique pour se préparer au prochain chapitre de l'exploration spatiale : envoyer des êtres humains vers des destinations plus lointaines comme la Lune et Mars. <br>

Cette simulation de mission a été menée à l'aide du Rover Juno de l'ASC : un rover robuste, tout-terrain. Ces tests sur le terrain ont été effectués en trois phases dans deux endroits : une carrière de pierres et le Terrain Analogique de l'ASC (également connu sous le nom de Mars Yard) au Québec. Le rover était opéré par des équipes basées à Saint-Hubert (Québec) et en Allemagne pour recréer la difficulté des communications longue distance. <br>

Ce projet était divisé comme suit : <br>
1. **LEAD/HOPE :** Ceci se concentrait sur le fait d'avoir des opérateurs formés effectuer des missions de retour d'échantillons. Cela a eu lieu sur cinq jours en octobre 2017 et quatre jours en juin 2019.
2. **Expérience de Collecte de Métriques du Rover LEAD (LRMGE) :** Cette partie de la mission avait six équipes en juin 2019 opérer le rover le long d'un itinéraire prédéfini pour collecter des métriques sur les performances de conduite du rover.
3. **Région Perpétuellement Ombragée LEAD (PSR) :** Enfin, cette partie se concentrait sur les tâches de conduite de rover sous des conditions d'éclairage sombre émulant les opérations dans une PSR.

L'ensemble de données analysé dans ce tutoriel provient de la phase LEAD (PSR) et a eu lieu en septembre 2019. Il est sous forme de rosbag et fournit l'imagerie, les données LiDAR, et la pose estimée du rover. <br>

Vous pouvez lire plus sur la mission ici : <br>
**Rover Juno :** https://www.asc-csa.gc.ca/eng/multimedia/search/image/7824<br>
**LEAD :** https://www.asc-csa.gc.ca/eng/rovers/mission-simulations/lunar-exploration-analogue-deployment.asp<br>
$\;\;\;\;\;\;\;\;$ https://www.hou.usra.edu/meetings/isairas2020fullpapers/pdf/5015.pdf

 ***

**Téléchargement de l'ensemble de données :** <br>
L'ensemble de données, sous forme de rosbag, peut être téléchargé depuis le Portail de Données Ouvertes : <br>
https://donnees-data.asc-csa.gc.ca/dataset/9151430-4v0p-4t5c-468vnhj714ao64

In [ ]:
#Required imports 
import imageio.v2 as imageio
import os, io, datetime, csv
import numpy as np
import rosbag, bagpy
import pandas as pd 
import open3d as o3d 
from sensor_msgs.point_cloud2 import read_points 
from matplotlib import pyplot as plt 
import geopandas, folium, base64
from folium import IFrame
import utm

***
### Ouverture d'un fichier ROSBAG
***

In [ ]:
#Opening rosbag file 
bagpy_bag = bagpy.bagreader('LEAD_delayed_2019-09-25-19-00-01-filtered.bag')
bag = rosbag.Bag('LEAD_delayed_2019-09-25-19-00-01-filtered.bag')

>Jetons un premier coup d'œil au contenu du rosbag.

In [ ]:
#List of topics & types of messages
print(bagpy_bag.topic_table)

>Nous commençons par extraire sensor_msgs/CameraInfo et les convertir en fichiers csv. Ceci contient les informations de calibration de caméra de la caméra centrale et gauche.

In [ ]:
#Decoding data by topic & converting to dataframe  (Camera Info & Localization)
Camera_info_centre_msg = bagpy_bag.message_by_topic('/delayed/artemisJr/centre/camera_info')
df_CAMERA_CENTRE = pd.read_csv(Camera_info_centre_msg) #Centre camera calibration

Camera_info_left_msg = bagpy_bag.message_by_topic('/delayed/artemisJr/left/camera_info')
df_CAMERA_LEFT = pd.read_csv(Camera_info_left_msg) #Left camera calibration

df_CAMERA_CENTRE.head(5)

*Un aperçu des 5 premières lignes de df_CAMERA_CENTRE*

***
### Extraction et visualisation des données de nuage de points (PCD)
***

Extrayons maintenant sensor_msgs/PointCloud2 et visualisons-les. Les données de nuage de points 3D proviennent du scanner 3D et sont utilisées pour la navigation (par exemple, évaluation du terrain, planification de parcours). Le nuage de points résultant est voxelisé et les points qui tombent sur le rover sont supprimés.

In [ ]:
#Extracting Point Cloud files from .bag file & converting them to csv files 
if __name__ == "__main__":
    path = './csv' #Path for csv folder 
    if not os.path.exists(path): #Check whether directory already exists, create otherwise
        os.mkdir(path)
        print("Folder %s created!" % path)
    else:
        print("Folder %s already exists" % path)
        
    for msg in bag:
        if  msg[0] == "/delayed/lowrate_scanner/points":
            print(msg[2])
            t = datetime.datetime.fromtimestamp(msg[2].to_sec()).strftime("%Y-%m-%d-%H_%M_%S")
            name = "csv/{0}-{1}.csv".format(msg[0].replace("/","_"),t)
            print("Saving to " + name)
            with open(name,"w") as f:
                # Read the points
                for point in read_points(msg[1]):
                    print("{0:.4f}, {1:.4f}, {2:.4f}".format(point[0],point[1],point[2]), file=f)
                f.close()

In [ ]:
path = os.getcwd() + '\\csv'
file_list = []
for root, dirs, files in os.walk(path):
    for file in files:
            file_list.append(os.path.join(root,file))       

In [ ]:
#Loading point cloud data and creating a point cloud object 
pcd_data = []
for file in file_list:
        name = file[-54:-4]
        data_csv = pd.read_csv(file)
        data_array = np.array(data_csv) #Converting to numpy array 
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(data_array)
        pcd_data.append(pcd)
        o3d.visualization.draw_geometries([pcd])
        #o3d.io.write_point_cloud(name + ".pcd", pcd)  #uncomment to save file 
        del pcd

*Visualisation d'un fichier de nuage de points*

In [ ]:
img = imageio.imread("1-PCD.png")
plt.imshow(img)
plt.axis("off")
plt.show()

>Plusieurs autres techniques sont potentiellement utiles dans la visualisation des données de nuages de points. Cela inclut des techniques telles que le développement d'une enveloppe convexe (l'enceinte de tous les points dans un espace), la visualisation de la grille voxel, ou la visualisation d'un octree, toutes montrées ci-dessous.

In [ ]:
i = 0
pcd_dbscan =  pcd_data[i] #update i as needed 

dist_list = []
#Creating a kd-tree for fast retreival of nearest neighbors
pcd_tree = o3d.geometry.KDTreeFlann(pcd_dbscan)

for i in range(len(np.asarray(pcd_dbscan.points))): 
    #For each point in the point cloud returns the 3 nearest neighbors distances (4 is used since also returns self)
    [k, _, dist] = pcd_tree.search_knn_vector_3d(pcd_dbscan.points[i], 4)

    for j in range(1, k):
        dist_list.append(dist[j])

#Sorting distances
dist_list.sort()

In [ ]:
#Plotting graph
dist_arr = np.asarray(dist_list)
plt.plot(dist_arr)

In [ ]:
min_points = 20
eps = 0.1 #from above graph

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd_dbscan.cluster_dbscan(eps = eps, min_points = min_points, print_progress=True)) # dbscan function
    
max_label = labels.max()

# setting a different color for each cluster with no clusters equaling black
colors = plt.get_cmap("gist_ncar")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0

pcd_dbscan.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd_dbscan])

In [ ]:
#Visualizing 
img = imageio.imread("2-DBSCAN.png")
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
#Other visualizations
i = 0
pcd_processed = pcd_data[i]

pcd_processed.scale(1 / np.max(pcd_data[i].get_max_bound() - pcd_data[i].get_min_bound()), center=pcd_processed.get_center())

# assigning a random color to each point in the point cloud 
pcd_processed.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(len(pcd_processed.points), 3)))

# determination of the convex hull including edges of the polygon
conv_hull, _  = pcd_processed.compute_convex_hull()
conv_hull.compute_vertex_normals() # edges
o3d.visualization.draw_geometries([conv_hull])

# displaying the voxel grid
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd_processed, voxel_size= 0.01) #voxel size correlates to grid
o3d.visualization.draw_geometries([voxel_grid])

# displaying octree
octree = o3d.geometry.Octree(max_depth=4) # depth specifies the level of detailed stored
octree.convert_from_point_cloud(pcd_processed, size_expand=0.01)
o3d.visualization.draw_geometries([octree])

*Le même fichier de nuage de points dans différentes visualisations*

In [ ]:
#Visualizing Point Cloud File 
fig = plt.figure()
Convex_Hull = imageio.imread("3-CONVEXHull.png")
Voxel_Grid = imageio.imread("4-VOXELGrid.png")
Octree = imageio.imread("5-OCTREE.png")

fig = plt.figure(figsize=(20, 15))
ax1 = fig.add_subplot(1,3,1)
plt.axis("off")
ax1.imshow(Convex_Hull)
ax2 = fig.add_subplot(1,3,2)
ax2.imshow(Voxel_Grid)
plt.axis("off")
ax3 = fig.add_subplot(1,3,3)
ax3.imshow(Octree)
plt.axis("off")

***
### Extraction et visualisation d'images compressées
***

Nous pouvons maintenant regarder les fichiers restants dans le rosbag qui sont de type sensor_msgs/CompressedImage. Il y a différents sujets dans le sac et vous pouvez remplacer le nom du sujet, selon les images que vous souhaitez extraire. Pour les besoins de ce tutoriel, nous allons extraire et visualiser des images compressées du sujet : */delayed/evo/left_polled/image_rect/compressed.* <br> Chaque message contient une image et un horodatage donc nous allons extraire les deux et les sauvegarder.

In [ ]:
#Create a list to save images and timestamps of desired topic. 
Image_Rect_Left_Polled = []
Image_Rect_Left_Polled_timestamps = []

In [ ]:
#Extracting raw compressed image data & saving to specific list (Images & TimeStamp)
if __name__ == "__main__":
    for msg in bag:
            if  msg[0] == "/delayed/evo/left_polled/image_rect/compressed": #Replace with desired topic
                timestamp = datetime.datetime.fromtimestamp(msg[2].to_sec()).strftime("%Y_%m_%d-%H_%M_%S") #Extract and transform timestamp
                Image_Rect_Left_Polled_timestamps.append(timestamp) #Replace list with corresponding topic list
                raw_data = io.BytesIO(msg[1].data) #Extract raw image data from .bag file
                image_data = imageio.v2.imread(raw_data)   
                Image_Rect_Left_Polled.append(image_data) #Replace list with corresponding topic
                

In [ ]:
#Visualize & save compressed images from list
path_image = './Images' #Path for new folder "Images"
    
#Check whether directory already exists
if not os.path.exists(path_image):
    os.mkdir(path_image)
    print("Folder %s created!" % path_image)
else:
    print("Folder %s already exists" % path_image)
        
os.chdir(path_image)

i = 0
for img in Image_Rect_Left_Polled: #replace list with desired topic
    plt.imshow(img, interpolation='nearest')
    plt.axis("off")
    timestamp = Image_Rect_Left_Polled_timestamps[i]
    plt.savefig("Image_Rect_Left_Polled "+ str(timestamp) +".jpg") #Change name to corresponding topic
    i += 1
print("This picture was taken on: " + timestamp)    
os.chdir('../') #Change directory back

>Un autre fichier dans le rosbag est de type geometry_msgs/PoseStamped. Il contient l'emplacement du rover ainsi que tous les horodatages. <br>
 Récupérons ces horodatages et formatons-les pour une meilleure lisibilité.

In [ ]:
#Get time stamps from all images and scans
timestamps_total = []
for msg in bag:
    if  msg[0] == "/delayed/trt_localization/pose":
        time = datetime.datetime.fromtimestamp(msg[2].to_sec()).strftime("%Y_%m_%d-%H_%M_%S")
        timestamps_total.append(time)

#Time stamps format updated and new csv file created 
location_csv = pd.read_csv(os.getcwd() + "\\LEAD_delayed_2019-09-25-19-00-01-filtered\\delayed-trt_localization-pose.csv") 
location_csv['TimeStamps'] = timestamps_total
location_csv.to_csv('Localization_TimeStamp_Updated.csv') 
location_csv.iloc[:,-8:].head(10) 

*Un aperçu des 10 premières lignes de colonnes sélectionnées*

***
### Cartographie du chemin du Rover Juno
***

Notre prochaine étape est de cartographier le chemin du Rover Juno sur le Mars Yard. Nous allons d'abord extraire les positions du rover du fichier csv et le convertir en format latitude et longitude.
> **Nous prenons les origines du Mars Yard comme (45.5182069858,-73.3939063839). Dans ce cas, nous projetons les positions x et y données, depuis l'origine.**

In [ ]:
#Converting mars yard origin to utm
mars_origin = utm.from_latlon(45.5182069858,-73.3939063839)
print(mars_origin)

In [ ]:
gdf = geopandas.GeoDataFrame(
    location_csv, geometry=geopandas.points_from_xy(location_csv["pose.position.x"], location_csv["pose.position.y"]))
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]

i = 0
for i in range(0,15951):
    test_x = geo_df_list[i][0]+625438.2348290744
    geo_df_list[i][0] = test_x
    test_y = 5041773.785209548 - geo_df_list[i][1]
    geo_df_list[i][1] = test_y
    i += 1

In [ ]:
#Converting all rover positions to latitude & longitude and converting csv file to dictonary
new_coord = []
lat = []
lon = []

i = 0
for i in range(0,15951):
    coord = utm.to_latlon(geo_df_list[i][0], geo_df_list[i][1], 18, northern = True)
    new_coord.append(coord)
    lat.append(coord[0])
    lon.append(coord[1])
    i += 1

location_csv['Lat'] = lat
location_csv['Lon'] = lon
location_csv.to_csv('Localization_Location_Updated.csv') 
location_csv

with open('Localization_Location_Updated.csv') as f:
    reader = csv.DictReader(f)
    Timestamp_csv_dict = [row for row in reader]

In [ ]:
location_csv.iloc[:,-3:].head(5) 

*Latitude et longitude du rover avec l'horodatage respectif*

>Puisque toutes nos images ont un horodatage attaché, nous pouvons faire une référence croisée avec l'horodatage dans notre Timestamp_csv_dict et obtenir une position du rover au moment où l'image a été capturée.

In [ ]:
#Cross references timestamps & returns image position (lat and lon)
def get_position(timestamp):
    for file in Timestamp_csv_dict:
        if timestamp == file['TimeStamps']:
            x_position = file['Lat']
            y_position = file['Lon']
            return (x_position, y_position)

>Nous avons maintenant toutes les informations nécessaires pour tracer une carte du chemin du rover.

In [ ]:
m = folium.Map(location=[45.5182069858,-73.3939063839], zoom_start=17)
centred_polyline=folium.PolyLine(locations=new_coord,weight=5)
m.add_child(centred_polyline)

In [ ]:
#Visualizing Map
img = imageio.imread("6-MAP.png")
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis("off")
plt.show()

>Nous pouvons aussi ajouter des fenêtres contextuelles pour montrer où était le rover au moment où une image spécifique (ou liste d'images) a été prise. Nous utilisons toujours des images du sujet */delayed/evo/left_polled/image_rect/compressed.*

In [ ]:
image_list = []
path_images = os.getcwd() +"\\Images"
for root, dirs, files in os.walk(path_images):
    for file in files:
            image_list.append(os.path.join(root,file)) 

In [ ]:
i = 0
for image in image_list:
    encoded = base64.b64encode(open(image,'rb').read())
    name = (image[-46:-4]).replace("_"," ") #Retrieving image name 
    html = (str(name) + '<img src="data:image/png;base64,{}">').format
    iframe = IFrame(html(encoded.decode('UTF-8')), width=650, height=550)
    popup = folium.Popup(iframe, max_width=650)
    loc = get_position(image[-23:-4]) #Cross-referencing timestamp to get lat & lon
    folium.Marker(location=[loc[0],loc[1]], tooltip=html, popup = popup, lazy = True,
              icon=folium.Icon(color = 'blue')).add_to(m)
    if len(image_list) > 7000:
        i += 500
    if len(image_list) > 1000 and len(image_list) < 7000:
        i += 100
    else:
        i += 1

In [ ]:
#Visualizing pop-ups added to map
img = imageio.imread("7-MAP_Images.png")
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis("off")
plt.show()

*Chaque image contient son sujet et la date-heure à laquelle elle a été capturée.*

In [ ]:
img = imageio.imread("6-MAP.jpeg")
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis("off")
plt.show()

*Image qui apparaît après avoir cliqué sur une icône*

In [ ]:
#Close rosbag 
bag.close()